In [6]:
!pip install psycopg2-binary pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 60.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 25.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 51.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 11.7 MB/s eta 0:00:00


In [2]:
topics = []
with open("./inputs/topics.txt", "r") as file:
    topics = file.read().split()
topics

['Sports-and-Outdoors',
 'Home-and-Kitchen',
 'Grocery-and-Gourmet-Food',
 'Automotive',
 'Software',
 'Office-Products',
 'Clothing-Shoes-and-Jewelry',
 'Baby',
 'Cell-Phones-and-Accessories',
 'Patio-Lawn-and-Garden',
 'Beauty',
 'Musical-Instruments',
 'Health-and-Personal-Care',
 'Pet-Supplies',
 'Electronics',
 'Tools-and-Home-Improvement',
 'Toys-and-Games',
 'Industrial-and-Scientific',
 'Video-Games',
 'Arts-Crafts-and-Sewing',
 'Appliances']

In [8]:
from collections import defaultdict
from psycopg2 import sql
import psycopg2
import pandas as pd

In [5]:
def get_column_names(conn, table):
    """
    Fetches and prints the column names of a specified table.
    """
    query = sql.SQL("""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = %s
        ORDER BY ordinal_position;
    """)
    cur = conn.cursor()
    cur.execute(query, (table,))
    columns = cur.fetchall()
    cur.close()
    return [col[0] for col in columns]

In [8]:
# QUERY ON ALL TOPIC NODES
dbname = "db"  # internal database name
username = "user"
password = "password"

table = "test"  # Name of table to query
count_topic = defaultdict(int)

for topic in topics:
    conn = psycopg2.connect(
                dbname=dbname,
                user=username,
                password=password,
                host=topic,
            )

    # Prepare and execute the query
    query = sql.SQL("SELECT id, question, answer, keywords FROM {}").format(sql.Identifier(table))
    try:
        with conn.cursor() as cur:
            cur.execute(query)
            rows = cur.fetchall()
            count_topic[topic] = len(rows)
    except psycopg2.errors.UndefinedTable:  # Handle table not found error
        print(f"Table '{table}' does not exist in database '{dbname}' on node '{topic}'")
        count_topic[topic] = 0  # Or handle the case as you see fit
    finally:
        conn.close()

# sort by desending frequency
count_topic = dict(sorted(count_topic.items(), key=lambda x: x[1], reverse=True))

print('Num Topic:', len(count_topic))
count_topic

Num Topic: 21


{'Electronics': 314263,
 'Home-and-Kitchen': 184439,
 'Sports-and-Outdoors': 146891,
 'Tools-and-Home-Improvement': 101088,
 'Automotive': 89923,
 'Cell-Phones-and-Accessories': 85865,
 'Health-and-Personal-Care': 80496,
 'Patio-Lawn-and-Garden': 59595,
 'Toys-and-Games': 51486,
 'Office-Products': 43608,
 'Beauty': 42422,
 'Pet-Supplies': 36607,
 'Baby': 28933,
 'Musical-Instruments': 23322,
 'Clothing-Shoes-and-Jewelry': 22068,
 'Arts-Crafts-and-Sewing': 21262,
 'Grocery-and-Gourmet-Food': 19538,
 'Video-Games': 13307,
 'Industrial-and-Scientific': 12136,
 'Software': 10636,
 'Appliances': 9011}

In [24]:
# CUSTOM QUERY ON SINGLE NODE
dbname = "db"  # internal database name
username = "user"
password = "password"

table = "test"  # Name of table to query
count_topic = defaultdict(int)

# test for topic[0]
topic = 'Automotive'
conn = psycopg2.connect(
        dbname=dbname,
        user=username,
        password=password,
        host=topic,
    )

query = sql.SQL("SELECT id, question, answer, keywords FROM {}").format(sql.Identifier(table))

try:
    with conn.cursor() as cur:
        header = get_column_names(conn, table)
        cur.execute(query)
        rows = cur.fetchall()
except psycopg2.errors.UndefinedTable:  # Handle table not found error
    print(f"Table '{table}' does not exist in database '{dbname}' on node '{topic}'")
finally:
    conn.close()

# for row in rows[:100]:
#     print(row)

node_df = pd.DataFrame(rows, columns=header[:-1])
node_df.sort_values('question')
node_df.head()


,id,question,answer,keywords
0,1,What is the most useful length to get?,at least 20 feet.......heres why....say you ha...,"[kw1, kw2, kw3, kw4, kw5]"
1,2,Are these cables made of copper or aluminum?,Coleman's website does indeed say copper equiv...,"[kw1, kw2, kw3, kw4, kw5]"
2,3,I bought the Red Extra Heavy Duty. Is that too...,"For jumper cables, you can never have too much...","[kw1, kw2, kw3, kw4, kw5]"
3,4,"Hi, Being 20ft 4gauge how heavy is this?",Not nearly heavy enough. I keep them under my ...,"[kw1, kw2, kw3, kw4, kw5]"
4,5,Do these cables come with a bag?,No,"[kw1, kw2, kw3, kw4, kw5]"


In [25]:
input_df = pd.read_csv('Automotive.csv')
input_df.sort_values('question')
input_df.head()

,question,answer,topic,index
0,!00% Sure this will work on a '14 Streetglide ...,Hell yea bro great and cheap got them on two o...,Automotive,82234
1,""" Item weight: 2.9 pounds""? What are these mad...","Amazon listed weight is wrong, shipping weight...",Automotive,56391
2,""" Programming this key requires 2 working keys...",Two fobs. If you have them (fobs) it is very e...,Automotive,72552
3,""" the threads are not standard pipe threads"". ...",U would have to get inch I'd hose a short piec...,Automotive,115992
4,"""13.5 x 25.5 x 21 inches"", is that the interio...",Mine is 10.5 x 11.5 X 22.25 in the bottom sect...,Automotive,72758
